In [6]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from scraping import init_driver, to_excel

In [7]:
driver = init_driver()
driver.get("http://www.randhawa.us/games/retailer/nyu.html")


In [8]:
# extract

driver.find_element(By.ID, "practiceButton").click()
time.sleep(2) 

data = driver.find_element(By.ID, "result-table").text
splitdata = data.split("\n")
columns = splitdata[0].split(" ")
values_split = [row.split(" ") for row in splitdata[1:]]

table = pd.DataFrame(values_split, columns=columns[:4])

#### Exploring strategies

In [9]:
def strategy(table, choices, chosenchoices):
    week = max(table['Week'].astype(int))
    if week > 1:
        prev_choice = chosenchoices[week-1]
        lift = (int(table['Sales'][week-1]) - int(table['Sales'][week-2])) / int(table['Sales'][week-2])
        lastchoice = max(chosenchoices)
        if lift < 0:
            print("Negative lift:", lift)
            return min(lastchoice+1, 3)
    return 0


In [10]:
attempts = 1000
all_data = []

for _ in range(attempts):
    chosenchoices = [0]
    driver.find_element(By.ID, "practiceButton").click()
    time.sleep(2)

    for i in range(15):
        data = driver.find_element(By.ID, "result-table").text
        splitdata = data.split("\n")
        columns = splitdata[0].split(" ")
        values_split = [row.split(" ") for row in splitdata[1:]]

        table = pd.DataFrame(values_split, columns=columns[:4])
        choice = strategy(table, [0, 1, 2, 3], chosenchoices)
        chosenchoices.append(choice)

        buttons = [
            driver.find_element(By.ID, "maintainButton"),
            driver.find_element(By.ID, "tenButton"),
            driver.find_element(By.ID, "twentyButton"),
            driver.find_element(By.ID, "fortyButton")
        ]
        buttons[choice].click()
        time.sleep(2)

        if choice == 3:
            data = driver.find_element(By.ID, "result-table").text
            splitdata = data.split("\n")
            columns = splitdata[0].split(" ")
            values_split = [row.split(" ") for row in splitdata[1:]]

            table = pd.DataFrame(values_split, columns=columns[:4])
            chosenchoices.extend([0] * (13 - i))
            break

    foresight = driver.find_element(By.CSS_SELECTOR, "div[style*='position:relative;top:10px;left:6px;']").text
    table['foresight'] = foresight
    table['Choice'] = chosenchoices
    table['diff'] = float(table['foresight'][0].split(": ")[3].replace("%", ""))
    table['revenue'] = float(table['foresight'][0].split(", ")[0].split(": ")[1].replace("$", "").replace(",", ""))
    table['perfect'] = float(table['foresight'][0].split(", ")[1].split(": ")[1].replace("$", "").replace(",", ""))
    
    all_data.append(table)

Negative lift: -0.375
Negative lift: -0.1623931623931624
Negative lift: -0.6693548387096774
Negative lift: -0.06611570247933884
Negative lift: -0.37222222222222223
Negative lift: -0.02654867256637168
Negative lift: -0.1513157894736842
Negative lift: -0.4217391304347826
Negative lift: -0.06779661016949153
Negative lift: -0.294478527607362
Negative lift: -0.45714285714285713
Negative lift: -0.189873417721519
Negative lift: -0.27722772277227725
Negative lift: -0.8064516129032258
Negative lift: -0.2009132420091324
Negative lift: -0.23529411764705882
Negative lift: -0.5247524752475248
Negative lift: -0.6551724137931034
Negative lift: -0.3352941176470588
Negative lift: -0.2648401826484018
Negative lift: -0.391156462585034
Negative lift: -0.0661764705882353
Negative lift: -0.015748031496062992
Negative lift: -0.30319148936170215
Negative lift: -0.20833333333333334
Negative lift: -0.45454545454545453
Negative lift: -0.42424242424242425
Negative lift: -0.45652173913043476
Negative lift: -0.1559

IndexError: list index out of range

In [11]:
# table result
table

,Week,Price,Sales,Remaining
0,1,60,161,1839
1,2,60,148,1691
2,3,54,228,1463
3,4,54,247,1216
4,5,54,248,968
5,6,54,251,717
6,7,54,262,455
7,8,54,234,221
8,9,48,221,0
9,10,48,0,0
